In [1]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:90% !important;}
div.cell.code_cell.rendered{width:100%;}
div.input_prompt{padding:0px;}
div.CodeMirror {font-family:Consolas; font-size:12pt;}
div.text_cell_render.rendered_html{font-size:12pt;}
div.output {font-size:12pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:12pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:12pt;padding:5px;}
table.dataframe{font-size:12px;}
</style>
"""))

<b><font size="6" color="red">ch15. 데이터베이스 연동</font></b>

# 1절. SQLite 데이터베이스 연결

- SQLite 데이터 베이스는 별도의 DBMS없이 SQL문을 이용해서 DB에 액세스할 수 있도록 만든 간단한 디스크 기반 DB(파일)
- C라이브러리
- SQLite는 프로토타입 생성시 사용
- 프로젝트 : 분석 &nbsp; → &nbsp; 설계 &nbsp; → &nbsp; 구현 &nbsp; → &nbsp; 테스트 &nbsp; → &nbsp; 고객에게 배포 &nbsp; → &nbsp; 유지보수
            (프로토타입:시제품)    완제품(Oracle, MySQL, Maria,...)
- [DB browser for SQLite](http://sqlitebrowser.org/dl/)

## 1.1 SQLite browser 설치 및 sqlite 패키지 load 

In [2]:
import sqlite3
sqlite3.sqlite_version # 이것만 버전보는방법이 다름

'3.40.1'

In [3]:
import pandas as pd
pd.__version__

'1.5.3'

## 1.2 데이터베이스 연결


(1) 데이터베이스 연결<br>
(2) SQL 전송 객체(cursor)<br>
(3) SQL 전송 & 결과 받기 (cursor.execute()이용)<br>
(4) cursor 해제 & 데이터베이스 연결객체 해제


In [4]:
# (1) DB 연결 : sqlite로 연결시 DB파일이 없으면 빈 DB파일 생성. 있으면 연결

conn = sqlite3.connect('data/ch15_example.db')
conn

In [5]:
# (2) SQL 전송 객체 생성(cursor 객체)

cursor = conn.cursor()
cursor

In [8]:
# (3) SQL 전송 & 결과 받기 (cursor.execute()이용)

cursor.execute('''
    CREATE TABLE MEMBER(
        NAME TEXT,
        AGE INT,
        EMAIL TEXT
    )
''')

In [7]:
cursor.execute('DROP TABLE MEMBER')

In [9]:
cursor.execute("INSERT INTO MEMBER VALUES ('홍길동',20,'h@h.com')")
print('수행결과 행수 :', cursor.rowcount) # 현재 트렌젝션에 있어서 커밋하지 않으면 새로고침해도 데이터가 들어가있지않음

수행결과 행수 : 1


In [10]:
sql = "INSERT INTO MEMBER VALUES('김길동', 30, 'K@K.com')"
cursor.execute(sql)
print('수행결과 행수 :', cursor.rowcount)
cursor.execute("INSERT INTO MEMBER VALUES ('이길동',25,'l@l.com')")
print('수행결과 행수 :', cursor.rowcount)

수행결과 행수 : 1
수행결과 행수 : 1


In [11]:
conn.commit() # 반대 : conn.rollback()

In [12]:
# select 전송 결과 : cursor가 가리킴

cursor.execute("SELECT * FROM MEMBER")

In [13]:
# insert, update, delete문 실행 결과 : cursor.rowcount
# select문 실행 결과 : 
    # fetchall() : 결과를 모두 받을 때 (튜플list)
    # fetchone() : 결과를 한행씩 받을 때 (튜플)
    # fetchmany(n) : 결과를 n행 받을 때(튜플list)
    
print(cursor.fetchall())

[('홍길동', 20, 'h@h.com'), ('김길동', 30, 'K@K.com'), ('이길동', 25, 'l@l.com')]


In [14]:
# 한번 소요된 cursor 객체는 다시 fetch될 수 없음

print(cursor.fetchall())

[]


In [15]:
# 한꺼번에 읽기 

cursor.execute("SELECT * FROM MEMBER ORDER BY AGE")
members = cursor.fetchall()
members

[('홍길동', 20, 'h@h.com'), ('이길동', 25, 'l@l.com'), ('김길동', 30, 'K@K.com')]

In [16]:
for member in members:
    print(member)

('홍길동', 20, 'h@h.com')
('이길동', 25, 'l@l.com')
('김길동', 30, 'K@K.com')


In [18]:
# 하나씩 읽기

cursor.execute("SELECT * FROM MEMBER ORDER BY AGE")
members = []
while True:
    member = cursor.fetchone()
    if member is None:
        print('데이터 끝')
        break
    members.append(member)

데이터 끝


In [19]:
members

[('홍길동', 20, 'h@h.com'), ('이길동', 25, 'l@l.com'), ('김길동', 30, 'K@K.com')]

In [20]:
# 최상위 n행 읽기

cursor.execute("SELECT * FROM MEMBER ORDER BY AGE")
for member in cursor.fetchmany(2):
    print(member)

('홍길동', 20, 'h@h.com')
('이길동', 25, 'l@l.com')


In [21]:
cursor.description

(('NAME', None, None, None, None, None, None),
 ('AGE', None, None, None, None, None, None),
 ('EMAIL', None, None, None, None, None, None))

In [25]:
class Member:
    'Member 테이블의 내용을 받을 객체 타입'
    def __init__(self, name, age, email):
        self.name = name
        self.age = age
        self.email = email
    def __str__(self):
        return f'{self.name}\t{self.age}\t{self.email}'
    
def to_member(*row): # 튜플매개변수를 받아 Member형 객체를 return
    return Member(row[0],row[1],row[2])

In [27]:
member = to_member('홍길동',20, 'h@h.com')
print(member)

홍길동	20	h@h.com


In [29]:
cursor.execute('SELECT * FROM MEMBER')
member_list = [] # sql문 수행한 결과를 담을 객체 list
members = cursor.fetchall() # 튜플 list
# print(members)
for member in members:
    member_list.append(to_member(*member))

In [30]:
for member in member_list:
    print(member)

홍길동	20	h@h.com
김길동	30	K@K.com
이길동	25	l@l.com


In [31]:
# (4) cursor 해제 & 데이터베이스 연결객체 해제

cursor.close()
conn.close()

## 1.3 SQL 구문에 파라미터 사용하기

- qmark(단점 : DB에 따라 불가한 경우 있음. ex.Oracle)
- named(추천)

In [32]:
conn = sqlite3.connect('data/ch15_example.db')
cursor = conn.cursor()
cursor.execute("SELECT * FROM MEMBER WHERE NAME IN ('홍길동','김길동')")
cursor.fetchall()

[('홍길동', 20, 'h@h.com'), ('김길동', 30, 'K@K.com')]

In [35]:
# 파라미터 사용하기 : qmark 방법

name1 = input('검색할 이름1?')
name2 = input('검색할 이름2?')
# cursor.execute("SELECT * FROM MEMBER WHERE NAME IN ('"+name1+"','"+name2+"')") # 가독성 구림
# cursor.execute(f"SELECT * FROM MEMBER WHERE NAME IN ('{name1}','{name2}')") # 가독성 조금 괜찮음
cursor.execute("SELECT * FROM MEMBER WHERE NAME IN (?,?)", (name1, name2))
cursor.fetchall()

검색할 이름1?홍길동
검색할 이름2?김길동


[('홍길동', 20, 'h@h.com'), ('김길동', 30, 'K@K.com')]

In [36]:
# 파라미터 사용하기 : named 방법

name1 = input('검색할 이름1?')
name2 = input('검색할 이름2?')
cursor.execute("SELECT * FROM MEMBER WHERE NAME IN (:name1,:name2)", {'name1':name1, 'name2':name2})
cursor.fetchall()

검색할 이름1?이길동
검색할 이름2?홍길동


[('홍길동', 20, 'h@h.com'), ('이길동', 25, 'l@l.com')]

In [38]:
# Member 테이블에 입력 (사용자로부터 이름, 나이, 메일을 입력받아 insert)
try:
    name = input('입력할 이름 :')
    age  = int(input('입력할 나이 :'))
except:
    print('유효하지 않은 나이를 입력한 경우, 18세로 초기화')
    age = 18
finally:
    email = input('입력할 메일은 :')
    
inputdata = {'name':name, 'age':age, 'email':email} # named방식에서 사용
inputdata2 = (name, age, email) # qmark방식에서 사용

sql = "INSERT INTO MEMBER VALUES (:name,:age,:email)"
sql2 = "INSERT INTO MEMBER VALUES (?,?,?)"

cursor.execute(sql, inputdata)
conn.commit()
if cursor.rowcount:
    print('저장완료')

입력할 이름 :홍홍홍
입력할 나이 :ㅁ
유효하지 않은 나이를 입력한 경우, 18세로 초기화
입력할 메일은 :hh@hh.com
저장완료


In [40]:
# Member 테이블에 입력 (사용자로부터 이름, 나이, 메일을 입력받아 insert)

try:
    name = input('입력할 이름 :')
    age  = int(input('입력할 나이 :'))
except:
    print('유효하지 않은 나이를 입력한 경우, 18세로 초기화')
    age = 18
finally:
    email = input('입력할 메일은 :')
    
newMember = Member(name,age,email)
# print(newMember)
# print(newMember.__dict__)

sql = "INSERT INTO MEMBER VALUES (:name,:age,:email)"
cursor.execute(sql, newMember.__dict__)
conn.commit()
if cursor.rowcount==1:
    print('입력성공')

입력할 이름 :황길동
입력할 나이 :27
입력할 메일은 :hwan@hgd.com
입력성공


In [42]:
# Member 테이블에 입력 (사용자로부터 이름, 나이, 메일을 입력받아 insert)

while True:
    try:
        name = input('입력할 이름(종료는 0) :')
        if name == '0':
            break;
        age  = int(input('입력할 나이 :'))
    except:
        print('유효하지 않은 나이를 입력한 경우, 18세로 초기화')
        age = 18
    email = input('입력할 메일은 :')

    newMember = Member(name,age,email)
    # print(newMember)
    # print(newMember.__dict__)

    sql = "INSERT INTO MEMBER VALUES (:name,:age,:email)"
    cursor.execute(sql, newMember.__dict__)
    if cursor.rowcount==1:
        print('입력성공')
conn.commit()

입력할 이름(종료는 0) :김길자
입력할 나이 :44
입력할 메일은 :
입력성공
입력할 이름(종료는 0) :홍홍자
입력할 나이 :ㅁ
유효하지 않은 나이를 입력한 경우, 18세로 초기화
입력할 메일은 :hhj@hh.com
입력성공
입력할 이름(종료는 0) :0


In [43]:
cursor.close()
conn.close()

# 2절. Oracle 데이터베이스 연결

- pip install cx_oracle (oracle 11g까지)
- pip install oracledb &nbsp;(oracle 12c부터)

In [44]:
import cx_Oracle
cx_Oracle.__version__

'8.3.0'

In [51]:
# conn객체 얻어오는 방법 1

conn = cx_Oracle.connect("scott", "tiger", "localhost:1521/xe")

# cursor객체 
cursor = conn.cursor()
sql = 'SELECT EMPNO "NO", ENAME, JOB, MGR, HIREDATE, SAL, COMM, DEPTNO FROM EMP'
cursor.execute(sql)
emp = cursor.fetchall()
print(emp)

[(7369, 'SMITH', 'CLERK', 7902, datetime.datetime(1980, 12, 17, 0, 0), 800.0, None, 20), (7499, 'ALLEN', 'SALESMAN', 7698, datetime.datetime(1981, 2, 20, 0, 0), 1600.0, 300.0, 30), (7521, 'WARD', 'SALESMAN', 7698, datetime.datetime(1981, 2, 22, 0, 0), 1250.0, 500.0, 30), (7566, 'JONES', 'MANAGER', 7839, datetime.datetime(1981, 4, 2, 0, 0), 2975.0, None, 20), (7654, 'MARTIN', 'SALESMAN', 7698, datetime.datetime(1981, 9, 28, 0, 0), 1250.0, 1400.0, 30), (7698, 'BLAKE', 'MANAGER', 7839, datetime.datetime(1981, 5, 1, 0, 0), 2850.0, None, 30), (7782, 'CLARK', 'MANAGER', 7839, datetime.datetime(1981, 6, 9, 0, 0), 2450.0, None, 10), (7788, 'SCOTT', 'ANALYST', 7566, datetime.datetime(1982, 12, 9, 0, 0), 3000.0, None, 20), (7839, 'KING', 'PRESIDENT', None, datetime.datetime(1981, 11, 17, 0, 0), 5000.0, None, 10), (7844, 'TURNER', 'SALESMAN', 7698, datetime.datetime(1981, 9, 8, 0, 0), 1500.0, 0.0, 30), (7876, 'ADAMS', 'CLERK', 7788, datetime.datetime(1983, 1, 12, 0, 0), 1100.0, None, 20), (7900, 

In [2]:
# conn객체 얻어오는 방법 2

import oracledb
oracledb.init_oracle_client()

#conn = oracledb.connect("scott/tiger@localhost:1521/xe")
conn = oracledb.connect(
    user='scott',
    password='tiger',
    host='localhost',
    port=1521,
    sid='xe'
)

# cursor객체 
cursor = conn.cursor()
sql = 'SELECT EMPNO "NO", ENAME, JOB, MGR, HIREDATE, SAL, COMM, DEPTNO FROM EMP'
cursor.execute(sql)
emp = cursor.fetchall() # 튜플 리스트
print(emp)

[(7369, 'SMITH', 'CLERK', 7902, datetime.datetime(1980, 12, 17, 0, 0), 800.0, None, 20), (7499, 'ALLEN', 'SALESMAN', 7698, datetime.datetime(1981, 2, 20, 0, 0), 1600.0, 300.0, 30), (7521, 'WARD', 'SALESMAN', 7698, datetime.datetime(1981, 2, 22, 0, 0), 1250.0, 500.0, 30), (7566, 'JONES', 'MANAGER', 7839, datetime.datetime(1981, 4, 2, 0, 0), 2975.0, None, 20), (7654, 'MARTIN', 'SALESMAN', 7698, datetime.datetime(1981, 9, 28, 0, 0), 1250.0, 1400.0, 30), (7698, 'BLAKE', 'MANAGER', 7839, datetime.datetime(1981, 5, 1, 0, 0), 2850.0, None, 30), (7782, 'CLARK', 'MANAGER', 7839, datetime.datetime(1981, 6, 9, 0, 0), 2450.0, None, 10), (7788, 'SCOTT', 'ANALYST', 7566, datetime.datetime(1982, 12, 9, 0, 0), 3000.0, None, 20), (7839, 'KING', 'PRESIDENT', None, datetime.datetime(1981, 11, 17, 0, 0), 5000.0, None, 10), (7844, 'TURNER', 'SALESMAN', 7698, datetime.datetime(1981, 9, 8, 0, 0), 1500.0, 0.0, 30), (7876, 'ADAMS', 'CLERK', 7788, datetime.datetime(1983, 1, 12, 0, 0), 1100.0, None, 20), (7900, 

In [4]:
import pandas as pd

emp_df = pd.DataFrame(emp)
emp_df

,0,1,2,3,4,5,6,7
0,7369,SMITH,CLERK,7902.0,1980-12-17,800.0,NaN,20
1,7499,ALLEN,SALESMAN,7698.0,1981-02-20,1600.0,300.0,30
2,7521,WARD,SALESMAN,7698.0,1981-02-22,1250.0,500.0,30
3,7566,JONES,MANAGER,7839.0,1981-04-02,2975.0,NaN,20
4,7654,MARTIN,SALESMAN,7698.0,1981-09-28,1250.0,1400.0,30
5,7698,BLAKE,MANAGER,7839.0,1981-05-01,2850.0,NaN,30
6,7782,CLARK,MANAGER,7839.0,1981-06-09,2450.0,NaN,10
7,7788,SCOTT,ANALYST,7566.0,1982-12-09,3000.0,NaN,20
8,7839,KING,PRESIDENT,NaN,1981-11-17,5000.0,NaN,10
9,7844,TURNER,SALESMAN,7698.0,1981-09-08,1500.0,0.0,30


In [5]:
# select문을 수행한 필드정보 (NO, ENAME, JOB, ~)

cursor.description

[('NO', <DbType DB_TYPE_NUMBER>, 5, None, 4, 0, False),
 ('ENAME', <DbType DB_TYPE_VARCHAR>, 10, 10, None, None, True),
 ('JOB', <DbType DB_TYPE_VARCHAR>, 9, 9, None, None, True),
 ('MGR', <DbType DB_TYPE_NUMBER>, 5, None, 4, 0, True),
 ('HIREDATE', <DbType DB_TYPE_DATE>, 23, None, None, None, True),
 ('SAL', <DbType DB_TYPE_NUMBER>, 11, None, 7, 2, True),
 ('COMM', <DbType DB_TYPE_NUMBER>, 11, None, 7, 2, True),
 ('DEPTNO', <DbType DB_TYPE_NUMBER>, 3, None, 2, 0, True)]

In [6]:
[description[0] for description in cursor.description]

['NO', 'ENAME', 'JOB', 'MGR', 'HIREDATE', 'SAL', 'COMM', 'DEPTNO']

In [7]:
emp_df.columns = [description[0] for description in cursor.description]
emp_df.head()

,NO,ENAME,JOB,MGR,HIREDATE,SAL,COMM,DEPTNO
0,7369,SMITH,CLERK,7902.0,1980-12-17,800.0,NaN,20
1,7499,ALLEN,SALESMAN,7698.0,1981-02-20,1600.0,300.0,30
2,7521,WARD,SALESMAN,7698.0,1981-02-22,1250.0,500.0,30
3,7566,JONES,MANAGER,7839.0,1981-04-02,2975.0,NaN,20
4,7654,MARTIN,SALESMAN,7698.0,1981-09-28,1250.0,1400.0,30


In [18]:
# 검색할 이름을 사용자에게 받아 해당 내용을 출력

ename = input('검색할 이름은?').upper()
print(ename)
sql = "SELECT * FROM EMP WHERE ENAME = :ename"
cursor.execute(sql, {'ename':ename})
emp = cursor.fetchall()
if emp:
    print(emp[0])
else:
    print('입력하신 이름의 데이터는 없습니다')

검색할 이름은?smith
SMITH
(7369, 'SMITH', 'CLERK', 7902, datetime.datetime(1980, 12, 17, 0, 0), 800.0, None, 20)


In [19]:
fieldnames = [descript[0] for descript in cursor.description]
fieldnames

['EMPNO', 'ENAME', 'JOB', 'MGR', 'HIREDATE', 'SAL', 'COMM', 'DEPTNO']

In [22]:
for fieldname, data in zip(fieldnames, emp[0]):
    print("{}:{}".format(fieldname, data if data is not None else ' -'))

EMPNO:7369
ENAME:SMITH
JOB:CLERK
MGR:7902
HIREDATE:1980-12-17 00:00:00
SAL:800.0
COMM: -
DEPTNO:20


In [23]:
cursor.close()
conn.close()

# 3절. MySQL 데이터베이스 연결

- pip install mysql-connector-python (공식connector)
- pip install PyMySQL (경량)

In [24]:
%pip install PyMySQL

     ---------------------------------------- 45.3/45.3 kB ? eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [25]:
import pymysql

In [30]:
conn = pymysql.connect(
    host='localhost',
    user='root',
    password='sun0p8i0p6y/',
    database='devdb',
    charset='utf8mb4' # 한글 깨짐 방지
)

cursor = conn.cursor()
sql = "SELECT * FROM PERSONAL"
cursor.execute(sql)
personal = cursor.fetchall()
personal

fieldnames = [descript[0] for descript in cursor.description]
fieldnames

pd.DataFrame(personal, columns=fieldnames)

,pno,pname,job,manager,startdate,pay,bonus,dno
0,1001,bill,president,NaN,1989-01-10,7000,NaN,10
1,1111,smith,manager,1001.0,1990-12-17,1000,NaN,10
2,1112,ally,salesman,1116.0,1991-02-20,1600,500.0,30
3,1113,word,salesman,1116.0,1992-02-24,1450,300.0,30
4,1114,james,manager,1001.0,1990-04-12,3975,NaN,20
5,1116,johnson,manager,1001.0,1991-05-01,3550,NaN,30
6,1118,martin,analyst,1111.0,1991-09-09,3450,NaN,10
7,1121,kim,clerk,1114.0,1990-12-08,4000,NaN,20
8,1123,lee,salesman,1116.0,1991-09-23,1200,0.0,30
9,1226,park,analyst,1111.0,1990-01-03,2500,NaN,10


In [31]:
cursor.close()
conn.close()

# 4절. 연습문제

## 실습형

- 회원가입 | 전체조회 | 이름찾기 | 메일삭제 | csv보내기 | 종료

### 0. 처음 실행

In [1]:
def load_cursor():
    global conn 
    import oracledb
    oracledb.init_oracle_client()
    conn = oracledb.connect('scott/tiger@localhost:1521/xe')
    #cursor = conn.cursor()
load_cursor()

### 1. 입력

In [2]:
# 1.입력

def fn1_insert_member_info():
    '입력받은 데이터를 member테이블에 입력'
    cursor = conn.cursor() 
    name = input('이름 :')
    tel = input('전화 :')
    email = input('이메일 :')
    
    try:
        age = int(input('나이 :'))
        if age>130:
            age=130
        elif age<0:
            age=0
    except:
        print('유효하지 않은 나이 입력시 나이는 0으로 초기화')
        age = 0
    
    try:
        grade = int(input('등급 :'))
        if grade<1:
            grade=1
        elif grade>5:
            grade=5
    except:
        print('유효하지 않은 등급 입력시 등급은 1등급으로 초기화')
        grade = 1
        
    etc = input('기타정보 :')
    
    sql = "INSERT INTO MEMBER VALUES(:name, :tel, :email, :age, :grade, :etc)"
    inputdata = {'name':name, 'tel':tel, 'email':email, 'age':age, 'grade':grade, 'etc':etc}
    cursor.execute(sql, inputdata)
    conn.commit()
    
    if cursor.rowcount==1:
        print(name, '님 가입 완료')
    cursor.close()

In [3]:
fn1_insert_member_info()

이름 :김길동
전화 :010-8888-8888
이메일 :g@g.com
나이 :22
등급 :-9
기타정보 :몰라
김길동 님 가입 완료


### 2. 전체 조회

In [18]:
# 2. 전체 출력

def fn2_print_members():
    'member 테이블의 내용을 데이터프레임으로 display'
    cursor = conn.cursor()
    sql = "SELECT * FROM MEMBER ORDER BY NAME"
    cursor.execute(sql)
    customer = cursor.fetchall()
    import pandas as pd
    df_customer = pd.DataFrame(customer, columns=[description[0] for description in cursor.description])
    display(df_customer)
    cursor.close()

In [19]:
fn2_print_members()

,NAME,TEL,EMAIL,AGE,GRADE,ETC
0,김길동,010-8888-8888,g@g.com,22,1,몰라
1,홍길동,010-9999-9999,H@H.COM,33,2,까칠해


### 3. 이름 조회

In [20]:
def fn3_search_member():
    '찾고자하는 이름을 input으로 받아 해당 내용을 데이터프레임으로 display'
    cursor = conn.cursor()
    name = input('찾으실 성함은 :')
    sql = "SELECT * FROM MEMBER WHERE NAME=:name"
    cursor.execute(sql, {'name':name})
    customer = cursor.fetchall()
    import pandas as pd
    df_customer = pd.DataFrame(customer, columns=[description[0] for description in cursor.description])
    display(df_customer)
    cursor.close()

In [21]:
fn3_search_member()

찾으실 성함은 :김길동


,NAME,TEL,EMAIL,AGE,GRADE,ETC
0,김길동,010-8888-8888,g@g.com,22,1,몰라


### 4. 메일 삭제

In [22]:
def fn4_delete_member():
    '삭제하고자 하는 메일을 input으로 받아 해당 행을 삭제'
    cursor = conn.cursor()
    email = input('삭제하실 메일은 :')
    sql = "DELETE FROM MEMBER WHERE EMAIL=:email"
    cursor.execute(sql, {'email':email})
    if cursor.rowcount==1:
        print('삭제 완료')
    else:
        print('존재하지 않는 메일입니다')
    conn.commit()
    cursor.close()

In [24]:
fn4_delete_member()

삭제하실 메일은 :g@g.com
삭제 완료


### 5. csv내보내기

In [27]:
def fn5_save_csv():
    'member테이블 내용을 csv로 내보내기'
    cursor = conn.cursor()
    sql = "SELECT * FROM MEMBER ORDER BY NAME"
    cursor.execute(sql)
    customer = cursor.fetchall()
    import pandas as pd
    df_customer = pd.DataFrame(customer, columns=[description[0] for description in cursor.description])
    filename = input('저장할 csv파일명')
    df_customer.to_csv(f'data/{filename}', encoding='utf-8', index=False)
    cursor.close()

In [28]:
fn5_save_csv()

저장할 csv파일명ch15예제


### 최종

In [ ]:
load_cursor()
while True:
    menu = input("1.회원입력 | 2.전체조회 | 3.이름찾기 | 4.메일삭제 | 5.CSV내보내기 | 0.종료")
    if menu=='1':
        fn1_insert_member_info()
    elif menu=='2':
        fn2_print_members()
    elif menu=='3':
        fn3_search_member()
    elif menu=='4':
        fn4_delete_member()
    elif menu=='5':
        fn5_save_csv()
    elif menu=='0':
        conn.close()
        break
    else:
        print('유효한 메뉴 번호를 입력해 주세요')